In [43]:
import pandas as pd
import numpy as np
import seaborn as sns
from wordcloud import WordCloud
import matplotlib.pyplot as plt
import nltk
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
import re
import string
from sklearn.model_selection import train_test_split


AttributeError: partially initialized module 'nltk' has no attribute 'data' (most likely due to a circular import)

#How to Approach 
1) using bag of word in feature extraction
2) trying NER; For sentiment analysis, adj. is most important
3) How to deal with those opinion and slang ??
4) How to deal with negation good != not good
5) splitting data into epoch cuz it's too big to put all in model at once

In [5]:
#import testset
df_iden = pd.read_csv("data_identification.csv")
df_iden


,tweet_id,identification
0,0x28cc61,test
1,0x29e452,train
2,0x2b3819,train
3,0x2db41f,test
4,0x2a2acc,train
...,...,...
1867530,0x227e25,train
1867531,0x293813,train
1867532,0x1e1a7e,train
1867533,0x2156a5,train


In [6]:
df_iden= df_iden[df_iden['identification'] == 'test']
df_test =df_iden
df_test.count() # test set

tweet_id          411972
identification    411972
dtype: int64

In [7]:
data_identification = pd.read_csv("data_identification.csv")
data_identification

,tweet_id,identification
0,0x28cc61,test
1,0x29e452,train
2,0x2b3819,train
3,0x2db41f,test
4,0x2a2acc,train
...,...,...
1867530,0x227e25,train
1867531,0x293813,train
1867532,0x1e1a7e,train
1867533,0x2156a5,train


In [8]:
df_test= df_test[df_test['identification'] == 'test']
df_test=df_test.dropna()
df_test

,tweet_id,identification
0,0x28cc61,test
3,0x2db41f,test
15,0x2466f6,test
23,0x23f9e9,test
31,0x1fb4e1,test
...,...,...
1867495,0x2c4dc2,test
1867496,0x31be7c,test
1867500,0x1ca58e,test
1867515,0x35c8ba,test


In [21]:
df_train = data_identification[data_identification['identification'] == 'train']
df_train

,tweet_id,identification
1,0x29e452,train
2,0x2b3819,train
4,0x2a2acc,train
5,0x2a8830,train
6,0x20b21d,train
...,...,...
1867530,0x227e25,train
1867531,0x293813,train
1867532,0x1e1a7e,train
1867533,0x2156a5,train


In [9]:
#import trainset
emo_df = pd.read_csv("emotion.csv")
tweets_df = pd.read_json(r'tweets_DM.json',lines=True)

In [10]:
tweets_df

,_score,_index,_source,_crawldate,_type
0,391,hashtag_tweets,"{'tweet': {'hashtags': ['Snapchat'], 'tweet_id...",2015-05-23 11:42:47,tweets
1,433,hashtag_tweets,"{'tweet': {'hashtags': ['freepress', 'TrumpLeg...",2016-01-28 04:52:09,tweets
2,232,hashtag_tweets,"{'tweet': {'hashtags': ['bibleverse'], 'tweet_...",2017-12-25 04:39:20,tweets
3,376,hashtag_tweets,"{'tweet': {'hashtags': [], 'tweet_id': '0x1cd5...",2016-01-24 23:53:05,tweets
4,989,hashtag_tweets,"{'tweet': {'hashtags': [], 'tweet_id': '0x2de2...",2016-01-08 17:18:59,tweets
...,...,...,...,...,...
1867530,827,hashtag_tweets,"{'tweet': {'hashtags': ['mixedfeeling', 'butim...",2015-05-12 12:51:52,tweets
1867531,368,hashtag_tweets,"{'tweet': {'hashtags': [], 'tweet_id': '0x29d0...",2017-10-02 17:54:04,tweets
1867532,498,hashtag_tweets,"{'tweet': {'hashtags': [], 'tweet_id': '0x2a6a...",2016-10-10 11:04:32,tweets
1867533,840,hashtag_tweets,"{'tweet': {'hashtags': [], 'tweet_id': '0x24fa...",2016-09-02 14:25:06,tweets


In [11]:
tweets_df.head()

,_score,_index,_source,_crawldate,_type
0,391,hashtag_tweets,"{'tweet': {'hashtags': ['Snapchat'], 'tweet_id...",2015-05-23 11:42:47,tweets
1,433,hashtag_tweets,"{'tweet': {'hashtags': ['freepress', 'TrumpLeg...",2016-01-28 04:52:09,tweets
2,232,hashtag_tweets,"{'tweet': {'hashtags': ['bibleverse'], 'tweet_...",2017-12-25 04:39:20,tweets
3,376,hashtag_tweets,"{'tweet': {'hashtags': [], 'tweet_id': '0x1cd5...",2016-01-24 23:53:05,tweets
4,989,hashtag_tweets,"{'tweet': {'hashtags': [], 'tweet_id': '0x2de2...",2016-01-08 17:18:59,tweets


In [12]:
#check for null value >> no null value
tweets_df.isnull() 

,_score,_index,_source,_crawldate,_type
0,False,False,False,False,False
1,False,False,False,False,False
2,False,False,False,False,False
3,False,False,False,False,False
4,False,False,False,False,False
...,...,...,...,...,...
1867530,False,False,False,False,False
1867531,False,False,False,False,False
1867532,False,False,False,False,False
1867533,False,False,False,False,False


In [13]:
tweets_df.shape

(1867535, 5)

In [24]:
#preprocessing trainset
data = tweets_df['_source'].tolist() # extract text coulum to be list obj
df = pd.DataFrame.from_dict(pd.json_normalize(data), orient='columns') # normalize data
tweets_DM= pd.concat([tweets_df, df], axis=1) # concat them again

tweets_DM

,_score,_index,_source,_crawldate,_type,tweet.hashtags,tweet.tweet_id,tweet.text
0,391,hashtag_tweets,"{'tweet': {'hashtags': ['Snapchat'], 'tweet_id...",2015-05-23 11:42:47,tweets,[Snapchat],0x376b20,"People who post ""add me on #Snapchat"" must be ..."
1,433,hashtag_tweets,"{'tweet': {'hashtags': ['freepress', 'TrumpLeg...",2016-01-28 04:52:09,tweets,"[freepress, TrumpLegacy, CNN]",0x2d5350,"@brianklaas As we see, Trump is dangerous to #..."
2,232,hashtag_tweets,"{'tweet': {'hashtags': ['bibleverse'], 'tweet_...",2017-12-25 04:39:20,tweets,[bibleverse],0x28b412,"Confident of your obedience, I write to you, k..."
3,376,hashtag_tweets,"{'tweet': {'hashtags': [], 'tweet_id': '0x1cd5...",2016-01-24 23:53:05,tweets,[],0x1cd5b0,Now ISSA is stalking Tasha 😂😂😂 <LH>
4,989,hashtag_tweets,"{'tweet': {'hashtags': [], 'tweet_id': '0x2de2...",2016-01-08 17:18:59,tweets,[],0x2de201,"""Trust is not the same as faith. A friend is s..."
...,...,...,...,...,...,...,...,...
1867530,827,hashtag_tweets,"{'tweet': {'hashtags': ['mixedfeeling', 'butim...",2015-05-12 12:51:52,tweets,"[mixedfeeling, butimTHATperson]",0x316b80,When you buy the last 2 tickets remaining for ...
1867531,368,hashtag_tweets,"{'tweet': {'hashtags': [], 'tweet_id': '0x29d0...",2017-10-02 17:54:04,tweets,[],0x29d0cb,I swear all this hard work gone pay off one da...
1867532,498,hashtag_tweets,"{'tweet': {'hashtags': [], 'tweet_id': '0x2a6a...",2016-10-10 11:04:32,tweets,[],0x2a6a4f,@Parcel2Go no card left when I wasn't in so I ...
1867533,840,hashtag_tweets,"{'tweet': {'hashtags': [], 'tweet_id': '0x24fa...",2016-09-02 14:25:06,tweets,[],0x24faed,"Ah, corporate life, where you can date <LH> us..."


In [17]:
#clean text
def clean_text(text):
    text = text.lower()
    text = re.sub('[%s]' % re.escape(string.punctuation), '', text)
    text = re.sub('\w*\d\w*', '', text)
    text = re.sub('\n', '', text)
    text = re.sub(r"[^A-Za-z0-9^,!.\/'+-=]", " ", text)
    text = re.sub(r"what's", "what is ", text)
    text = re.sub(r"\'s", " ", text)
    text = re.sub(r"\'ve", " have ", text)
    text = re.sub(r"n't", " not ", text)
    text = re.sub(r"i'm", "i am ", text)
    text = re.sub(r"\'re", " are ", text)
    text = re.sub(r"\'d", " would ", text)
    text = re.sub(r"\'ll", " will ", text)
    text = re.sub(r",", " ", text)
    text = re.sub(r"\.", " ", text)
    text = re.sub(r"!", " ! ", text)
    text = re.sub(r"\/", " ", text)
    text = re.sub(r"\^", " ^ ", text)
    text = re.sub(r"\+", " + ", text)
    text = re.sub(r"\-", " - ", text)
    text = re.sub(r"\=", " = ", text)
    text = re.sub(r"'", " ", text)
    text = re.sub(r"(\d+)(k)", r"\g<1>000", text)
    text = re.sub(r":", " : ", text)
    text = re.sub(r" e g ", " eg ", text)
    text = re.sub(r" b g ", " bg ", text)
    text = re.sub(r" u s ", " american ", text)
    text = re.sub(r"\0s", "0", text)
    text = re.sub(r" 9 11 ", "911", text)
    text = re.sub(r"e - mail", "email", text)
    text = re.sub(r"j k", "jk", text)
    text = re.sub(r"\s{2,}", " ", text)
    text = re.sub(r"<LH>", " ", text)
    return text

In [18]:
tweets_DM['text_clean'] = tweets_DM['tweet.text'].apply(clean_text)
tweets_DM[0::10][:][0:40]

,_score,_index,_source,_crawldate,_type,tweet.hashtags,tweet.tweet_id,tweet.text,text_clean
0,391,hashtag_tweets,"{'tweet': {'hashtags': ['Snapchat'], 'tweet_id...",2015-05-23 11:42:47,tweets,[Snapchat],0x376b20,"People who post ""add me on #Snapchat"" must be ...",people who post add me on snapchat must be deh...
10,631,hashtag_tweets,"{'tweet': {'hashtags': ['SSM', 'gender', 'dive...",2016-08-05 21:15:36,tweets,"[SSM, gender, diversity]",0x359db9,The #SSM debate; <LH> (a manufactured fantasy ...,the ssm debate lh a manufactured fantasy used ...
20,705,hashtag_tweets,"{'tweet': {'hashtags': ['ThankULord'], 'tweet_...",2017-07-22 12:30:48,tweets,[ThankULord],0x2c49da,Giving thanks to my lord and savior for allowi...,giving thanks to my lord and savior for allowi...
30,104,hashtag_tweets,"{'tweet': {'hashtags': ['GodsPlan', 'GodsWork'...",2015-10-10 14:33:26,tweets,"[GodsPlan, GodsWork]",0x2939d5,"God woke you up, now chase the day #GodsPlan #...",god woke you up now chase the day godsplan god...
40,663,hashtag_tweets,"{'tweet': {'hashtags': ['hangry', 'needfood'],...",2016-10-19 04:57:08,tweets,"[hangry, needfood]",0x2ac7bd,Hangry and ready to snap. #hangry <LH> #needfo...,hangry and ready to snap hangry lh needfood
50,641,hashtag_tweets,"{'tweet': {'hashtags': [], 'tweet_id': '0x20ca...",2015-09-15 20:21:03,tweets,[],0x20caf5,@peplamb There is no <LH> if @realDonaldTrump ...,peplamb there is no lh if realdonaldtrump is p...
60,100,hashtag_tweets,"{'tweet': {'hashtags': ['scared'], 'tweet_id':...",2016-10-01 17:49:08,tweets,[scared],0x2bf9ce,Saw the county's bitch today. Pulled up next t...,saw the countys bitch today pulled up next to ...
70,406,hashtag_tweets,"{'tweet': {'hashtags': [], 'tweet_id': '0x2c97...",2015-07-22 04:43:33,tweets,[],0x2c9756,<LH> <LH> <LH> Nimrodel beckons and geranosaur...,lh lh lh nimrodel beckons and geranosaurus baf...
80,394,hashtag_tweets,"{'tweet': {'hashtags': [], 'tweet_id': '0x37e9...",2017-08-01 05:05:31,tweets,[],0x37e94f,Happy Thanksgiving <LH> 🦃🦃,happy thanksgiving lh
90,229,hashtag_tweets,"{'tweet': {'hashtags': ['Shooting', 'Valley', ...",2015-02-05 13:06:27,tweets,"[Shooting, Valley, Girls, like, saying, Really]",0x212d86,We should take #Shooting #Valley #Girls for #l...,we should take shooting valley girls for like ...


In [25]:
#preprocessing trainset
tweets_DM_train = tweets_DM.merge(df_train, how='inner',left_on = 'tweet.tweet_id', right_on = 'tweet_id')

In [30]:
tweets_DM_train# train dataset with text clean

tweets_DM_train['text_clean'] = tweets_DM_train['tweet.text'].apply(clean_text)
tweets_DM_train[0::10][:][0:40]


,_score,_index,_source,_crawldate,_type,tweet.hashtags,tweet.tweet_id,tweet.text,tweet_id,identification,text_clean
0,391,hashtag_tweets,"{'tweet': {'hashtags': ['Snapchat'], 'tweet_id...",2015-05-23 11:42:47,tweets,[Snapchat],0x376b20,"People who post ""add me on #Snapchat"" must be ...",0x376b20,train,people who post add me on snapchat must be deh...
10,887,hashtag_tweets,{'tweet': {'hashtags': ['justgradstudentthings...,2015-08-11 16:22:15,tweets,"[justgradstudentthings, ecology]",0x37a0a9,You know you research butterflies when predict...,0x37a0a9,train,you know you research butterflies when predict...
20,187,hashtag_tweets,"{'tweet': {'hashtags': [], 'tweet_id': '0x2cab...",2015-05-14 14:48:43,tweets,[],0x2cab19,I’ll never be half the man Pat Tillman was. <LH>,0x2cab19,train,i ll never be half the man pat tillman was lh
30,125,hashtag_tweets,"{'tweet': {'hashtags': ['24hours', 'iPhoneX', ...",2016-01-21 05:22:22,tweets,"[24hours, iPhoneX, Windians, clevelandindians,...",0x1ea076,Follow for <LH> next #24hours ? #iPhoneX #Wind...,0x1ea076,train,follow for lh next iphonex windians lh clevela...
40,786,hashtag_tweets,"{'tweet': {'hashtags': ['way', 'truth', 'life'...",2017-10-17 15:00:14,tweets,"[way, truth, life]",0x274427,"<LH> is the #way, the #truth, and the #life",0x274427,train,lh is the way the truth and the life
50,939,hashtag_tweets,"{'tweet': {'hashtags': [], 'tweet_id': '0x2a46...",2016-03-12 16:24:53,tweets,[],0x2a468e,@sydbell1 @strut4cancer £50 donates to Macmill...,0x2a468e,train,donates to macmillan lh
60,372,hashtag_tweets,"{'tweet': {'hashtags': [], 'tweet_id': '0x2498...",2017-10-14 09:51:30,tweets,[],0x2498cf,I want that Frank Ocean phone cover <LH>,0x2498cf,train,i want that frank ocean phone cover lh
70,468,hashtag_tweets,"{'tweet': {'hashtags': ['suicide'], 'tweet_id'...",2017-07-28 19:45:18,tweets,[suicide],0x2dd4e0,"Anyone wants to talk about anything, hit me up...",0x2dd4e0,train,anyone wants to talk about anything hit me up ...
80,823,hashtag_tweets,"{'tweet': {'hashtags': ['STOP', 'CHANGE', 'DOU...",2016-10-20 17:22:40,tweets,"[STOP, CHANGE, DOUBT]",0x2e8159,DON'T #STOP THE PROGRESSIVE #CHANGE <LH> IS MA...,0x2e8159,train,dont stop the progressive change lh is making ...
90,234,hashtag_tweets,"{'tweet': {'hashtags': ['givehope', 'givetarir...",2015-09-20 03:07:36,tweets,"[givehope, givetariro]",0x31dcef,Few things in the world are more powerful than...,0x31dcef,train,few things in the world are more powerful than...


In [34]:
tweets_DM_train.shape

(1455563, 11)

In [27]:
tweets_DM_test = tweets_DM.merge(df_test, how='inner',left_on = 'tweet.tweet_id', right_on = 'tweet_id')

In [32]:
# test dataset with text clean
tweets_DM_test['text_clean'] = tweets_DM_test['tweet.text'].apply(clean_text)
tweets_DM_test[0::10][:][0:40]


,_score,_index,_source,_crawldate,_type,tweet.hashtags,tweet.tweet_id,tweet.text,tweet_id,identification,text_clean
0,232,hashtag_tweets,"{'tweet': {'hashtags': ['bibleverse'], 'tweet_...",2017-12-25 04:39:20,tweets,[bibleverse],0x28b412,"Confident of your obedience, I write to you, k...",0x28b412,test,confident of your obedience i write to you kno...
10,112,hashtag_tweets,"{'tweet': {'hashtags': [], 'tweet_id': '0x25dc...",2015-12-07 06:18:42,tweets,[],0x25dcd8,Nothing like the delicious taste of coffee and...,0x25dcd8,test,nothing like the delicious taste of coffee and...
20,540,hashtag_tweets,"{'tweet': {'hashtags': [], 'tweet_id': '0x245e...",2016-07-21 05:34:13,tweets,[],0x245e5b,@realDonaldTrump It's been on the news all day...,0x245e5b,test,realdonaldtrump its been on the news all day u...
30,369,hashtag_tweets,"{'tweet': {'hashtags': ['PuertoRico'], 'tweet_...",2016-08-01 18:10:02,tweets,[PuertoRico],0x1f0f0d,@realDonaldTrump Other places have been hit by...,0x1f0f0d,test,realdonaldtrump other places have been hit by ...
40,573,hashtag_tweets,"{'tweet': {'hashtags': ['GCCC', 'SCMP', 'yyc',...",2015-08-02 22:33:24,tweets,"[GCCC, SCMP, yyc, certification, communications]",0x3186fc,"Well, I did it. I registered to write the @IAB...",0x3186fc,test,well i did it i registered to write the iabc g...
50,40,hashtag_tweets,"{'tweet': {'hashtags': [], 'tweet_id': '0x2f07...",2015-07-09 10:15:34,tweets,[],0x2f07b4,When your husband forgots to turn his alarm c...,0x2f07b4,test,when your husband forgots to turn his alarm cl...
60,929,hashtag_tweets,"{'tweet': {'hashtags': ['bored', 'dm'], 'tweet...",2015-02-02 02:35:40,tweets,"[bored, dm]",0x2940be,Someone anyone hit my DM ’s... I’m bored #bo...,0x2940be,test,someone anyone hit my dm s i m bored bored dm lh
70,627,hashtag_tweets,"{'tweet': {'hashtags': [], 'tweet_id': '0x2041...",2015-06-27 11:05:21,tweets,[],0x20410d,Just got invited to a kids birthday party by t...,0x20410d,test,just got invited to a kids birthday party by t...
80,470,hashtag_tweets,"{'tweet': {'hashtags': [], 'tweet_id': '0x273e...",2015-09-22 07:02:01,tweets,[],0x273eb4,@CumBubbleJesus Oddly enough I couldn't rememb...,0x273eb4,test,cumbubblejesus oddly enough i couldnt remember...
90,130,hashtag_tweets,"{'tweet': {'hashtags': ['FiEurope2017'], 'twee...",2017-09-24 23:35:30,tweets,[FiEurope2017],0x2d46e2,#FiEurope2017 is about to open and we can't wa...,0x2d46e2,test,is about to open and we cant wait come say hi...


In [33]:
tweets_DM_test.shape

(411972, 11)

# try cleaning text with basic package <Hope it works>

not working!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!    
    
import re
import string

def clean_text(text):
    
    ## Remove puncuation
    text = text.translate(string.punctuation)
    
    ## Convert words to lower case and split them
    text = text.lower().split()
    
    ## Remove stop words
    stops = set(stopwords.words("english"))
    text = [w for w in text if not w in stops and len(w) >= 3]
    
    text = " ".join(text)

    # Clean the text
    text = re.sub(r"[^A-Za-z0-9^,!.\/'+-=]", " ", text)
    text = re.sub(r"what's", "what is ", text)
    text = re.sub(r"\'s", " ", text)
    text = re.sub(r"\'ve", " have ", text)
    text = re.sub(r"n't", " not ", text)
    text = re.sub(r"i'm", "i am ", text)
    text = re.sub(r"\'re", " are ", text)
    text = re.sub(r"\'d", " would ", text)
    text = re.sub(r"\'ll", " will ", text)
    text = re.sub(r",", " ", text)
    text = re.sub(r"\.", " ", text)
    text = re.sub(r"!", " ! ", text)
    text = re.sub(r"\/", " ", text)
    text = re.sub(r"\^", " ^ ", text)
    text = re.sub(r"\+", " + ", text)
    text = re.sub(r"\-", " - ", text)
    text = re.sub(r"\=", " = ", text)
    text = re.sub(r"'", " ", text)
    text = re.sub(r"(\d+)(k)", r"\g<1>000", text)
    text = re.sub(r":", " : ", text)
    text = re.sub(r" e g ", " eg ", text)
    text = re.sub(r" b g ", " bg ", text)
    text = re.sub(r" u s ", " american ", text)
    text = re.sub(r"\0s", "0", text)
    text = re.sub(r" 9 11 ", "911", text)
    text = re.sub(r"e - mail", "email", text)
    text = re.sub(r"j k", "jk", text)
    text = re.sub(r"\s{2,}", " ", text)
    text = re.sub(r"LH", " ", text)
    
    text = text.split()
    stemmer = SnowballStemmer('english')
    stemmed_words = [stemmer.stem(word) for word in text]
    text = " ".join(stemmed_words)

    return text

AttributeError: partially initialized module 'nltk' has no attribute 'data' (most likely due to a circular import)

In [38]:
#tokenizer - - using bag of words BOW 
from sklearn.feature_extraction.text import CountVectorizer

# build analyzers (bag-of-words)
BOW_200 = CountVectorizer(max_features=200, tokenizer=nltk.word_tokenize) 

# apply analyzer to training data
BOW_200.fit(tweets_DM_train['text_clean'])

train_data_BOW_features_200 = BOW_200.transform(tweets_DM_train['text_clean'])

## check dimension
train_data_BOW_features_200.shape

NameError: name 'nltk' is not defined

In [ ]:
train_data_BOW_features_200.shape

In [ ]:
# HOW TO SOLVE THIS PROB
#.toarray() is used to convert list to an array; what if the dataset is too big
# change max_featres from 500 to 250 and it works!!
train_data_BOW_features_200.toarray() 

In [ ]:
feature_names_200 = BOW_200.get_feature_names_out()
feature_names_200[100:110]

In [ ]:
# create y_train (resulr) 

y_train = tweets_DM ['emotion']
y_train

In [ ]:
train_data_BOW_features_200.shape

In [ ]:
#prepare test set
df_test

In [ ]:
data = tweets_df['_source'].tolist() # extract text coulum to be list obj
df = pd.DataFrame.from_dict(pd.json_normalize(data), orient='columns') # normalize data
tweets_DM_test= pd.concat([tweets_df, df], axis=1) # concat them again

In [ ]:
tweets_DM_test

In [ ]:
#creat test set
testset = df_test.merge(tweets_DM_test, how='inner',left_on='tweet_id', right_on='tweet.tweet_id')
testset=testset['tweet.text']

In [ ]:
testset

In [ ]:
testset = testset['tweet.text'].apply(clean_text)
testset[0::10][:][0:40]

In [ ]:
#tokenize test set
# build analyzers (bag-of-words)
test_BOW_200 = CountVectorizer(max_features=200, tokenizer=nltk.word_tokenize) 

# apply analyzer to training data
test_BOW_200.fit(testset['tweet.text'])


test_BOW_200_features = test_BOW_200.transform(testset['tweet.text'])

## check dimension
test_BOW_200_features





In [ ]:
test_BOW_200_features.toarray() 



#tokenize test set
# build analyzers (bag-of-words)
y_train2 = CountVectorizer(max_features=200, tokenizer=nltk.word_tokenize) 

# apply analyzer to training data
y_train2.fit(y_train)

y_train2_features = test_BOW_200.transform(y_train)

## check dimension
y_train2



y_train2.toarray() 

In [ ]:
#create mode -- naivebay
from sklearn.naive_bayes import MultinomialNB, GaussianNB, BernoulliNB
naivebay = BernoulliNB()
naive=naivebay.fit(train_data_BOW_features_200, y_train)



In [ ]:
result = naive.predict(test_BOW_200_features)

In [ ]:
result

In [ ]:
df_test['emotion'] = result

In [ ]:
df_test

In [ ]:
df_test[['tweet_id','emotion']].to_csv('sixth')